In [2]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pickle

class SkeletonDataset(Dataset):
    """
    Custom Dataset cho dữ liệu skeleton.
    Đọc dữ liệu từ file .npy và label từ file .pkl,
    sau đó định hình lại cho phù hợp với mô hình RNN/GRU.
    """
    def __init__(self, data_path, label_path):
        # Tải label
        with open(label_path, 'rb') as f:
            self.labels = pickle.load(f)

        # Tải dữ liệu và chuyển sang tensor
        # Dữ liệu gốc có shape (N, C, T, V, M)
        raw_data = np.load(data_path)
        self.data = torch.from_numpy(raw_data).float()

        # Thông tin shape
        self.N, self.C, self.T, self.V, self.M = self.data.shape
        print(f"Loaded data from {data_path} with shape: ({self.N}, {self.C}, {self.T}, {self.V}, {self.M})")


    def __len__(self):
        return self.N

    def __getitem__(self, index):
        # Lấy dữ liệu và label tại index
        sample_data = self.data[index] # Shape: (C, T, V, M)
        label = self.labels[index]

        # Thay đổi shape để phù hợp với GRU
        # Chuyển (C, T, V, M) -> (T, C, V, M)
        sample_data = sample_data.permute(1, 0, 2, 3)

        # Làm phẳng các chiều C, V, M thành một vector đặc trưng duy nhất cho mỗi bước thời gian T
        # Shape sau khi làm phẳng: (T, C * V * M)
        # Ví dụ: (20, 3 * 61 * 1) = (20, 183)
        flattened_data = sample_data.reshape(self.T, -1)

        return flattened_data, torch.tensor(label, dtype=torch.long)

In [4]:
import random
def set_seed(seed_value):
    random.seed(seed_value) # Python's random module
    np.random.seed(seed_value) # NumPy
    torch.manual_seed(seed_value) # PyTorch on CPU
    torch.cuda.manual_seed_all(seed_value) # PyTorch on all GPUs (if available)
    
    # Cấu hình cho thuật toán cuDNN (thường được sử dụng bởi PyTorch trên GPU)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    print(f"Global seed set to {seed_value}")

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdvancedGRUModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(AdvancedGRUModel, self).__init__()

        # --- Lớp GRU ---
        # Keras: GRU(128, return_sequences=True, activation='relu')
        # PyTorch: nn.GRU không hỗ trợ trực tiếp activation='relu'. 
        # Chúng ta sẽ sử dụng nn.GRU tiêu chuẩn và thảo luận về điểm này bên dưới.
        self.gru1 = nn.GRU(input_size=input_size, hidden_size=128, batch_first=True)
        self.dropout1 = nn.Dropout(0)

        self.gru2 = nn.GRU(input_size=128, hidden_size=128, batch_first=True)
        self.dropout2 = nn.Dropout(0)

        self.gru3 = nn.GRU(input_size=128, hidden_size=256, batch_first=True)
        self.dropout3 = nn.Dropout(0)
        
        self.gru4 = nn.GRU(input_size=256, hidden_size=512, batch_first=True)
        self.dropout4 = nn.Dropout(0)

        # Lớp GRU cuối cùng, tương đương return_sequences=False
        self.gru5 = nn.GRU(input_size=512, hidden_size=512, batch_first=True)

        # --- Khối Classifier (MLP) ---
        # Sử dụng nn.Sequential để nhóm các lớp Dense lại cho gọn gàng
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(512),       # Tương đương BatchNormalization() sau GRU
            nn.Linear(512, 512),
            nn.ReLU(),                 # Tương đương activation='relu'
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes) # Lớp output, không có softmax (xem giải thích)
        )

    def forward(self, x):
        # Luồng dữ liệu qua các lớp GRU
        # self.gru(x) trả về (output, h_n)
        # output có shape (batch, seq_len, hidden_size)
        # h_n là hidden state cuối cùng
        
        # return_sequences=True
        out, _ = self.gru1(x)
        out = self.dropout1(out)
        
        out, _ = self.gru2(out)
        out = self.dropout2(out)

        out, _ = self.gru3(out)
        out = self.dropout3(out)

        out, _ = self.gru4(out)
        out = self.dropout4(out)

        # Lớp GRU cuối, tương đương return_sequences=False
        out, _ = self.gru5(out)
        
        # Lấy output của bước thời gian cuối cùng
        out = out[:, -1, :] # Shape: (batch, hidden_size)

        # Cho qua khối classifier
        out = self.classifier(out)
        
        return out

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint_path = '/workspace/data/log/hdf5/ced_gru_model.pth'

# 2. Khởi tạo model với đúng input_size và num_classes
input_size  = 3 * 61 * 1    # giữ nguyên như lúc train
num_classes = 2139          # số lớp của bạn
model = AdvancedGRUModel(input_size=input_size, num_classes=num_classes)
model.to(device)

# 1. Load checkpoint (có thể là dict hoặc có key 'state_dict')
ckpt = torch.load(checkpoint_path, map_location=device)
pretrained_dict = ckpt.get('state_dict', ckpt)

# 2. Lấy state_dict hiện tại của model
model_dict = model.state_dict()

# 3. Lọc ra những keys vừa tồn tại, vừa cùng shape
filtered_dict = {
    k: v for k, v in pretrained_dict.items()
    if k in model_dict and v.shape == model_dict[k].shape
}

# 4. Update weights và load
model_dict.update(filtered_dict)
model.load_state_dict(model_dict)

# 5. Chuyển sang inference mode
model.eval()


AdvancedGRUModel(
  (gru1): GRU(183, 128, batch_first=True)
  (dropout1): Dropout(p=0, inplace=False)
  (gru2): GRU(128, 128, batch_first=True)
  (dropout2): Dropout(p=0, inplace=False)
  (gru3): GRU(128, 256, batch_first=True)
  (dropout3): Dropout(p=0, inplace=False)
  (gru4): GRU(256, 512, batch_first=True)
  (dropout4): Dropout(p=0, inplace=False)
  (gru5): GRU(512, 512, batch_first=True)
  (classifier): Sequential(
    (0): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Linear(in_features=256, out_features=256, bias=True)
    (6): ReLU()
    (7): Linear(in_features=256, out_features=128, bias=True)
    (8): ReLU()
    (9): Linear(in_features=128, out_features=64, bias=True)
    (10): ReLU()
    (11): Linear(in_features=64, out_features=2139, bias=True)
  )
)

In [7]:
DATA_DIR = '/workspace/data/npy_splits_new'

# Chỉ cần loader cho tập bạn muốn đánh giá
test_dataset = SkeletonDataset(
    f'{DATA_DIR}/test_data.npy',
    f'{DATA_DIR}/test_label.pkl'
)
test_loader = DataLoader(
    test_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

Loaded data from /workspace/data/npy_splits_new/test_data.npy with shape: (2139, 3, 24, 61, 1)


In [8]:
import torch.nn as nn
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()

def inference_and_report(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Inference"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)

            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(dataloader.dataset)
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean()

    print(f"\nInference Results → Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}\n")
    print("=== Classification Report ===")
    print(classification_report(all_labels, all_preds, digits=4))
    print("\n=== Confusion Matrix (shape {}) ===".format(
        confusion_matrix(all_labels, all_preds).shape
    ))
    # Nếu bạn không cần in ma trận đầy đủ, có thể bỏ phần này.

# Cuối cùng, chỉ cần gọi:
inference_and_report(model, test_loader, criterion, device)


Inference: 100%|██████████| 9/9 [00:00<00:00, 25.21it/s]


Inference Results → Loss: 92.6517, Acc: 0.2950

=== Classification Report ===
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         1
           1     0.0000    0.0000    0.0000         1
           2     0.0000    0.0000    0.0000         1
           3     0.0000    0.0000    0.0000         1
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         1
           6     0.0000    0.0000    0.0000         1
           7     0.0000    0.0000    0.0000         1
           8     1.0000    1.0000    1.0000         1
           9     0.0000    0.0000    0.0000         1
          10     0.0000    0.0000    0.0000         1
          11     0.0000    0.0000    0.0000         1
          12     0.0000    0.0000    0.0000         1
          13     0.0000    0.0000    0.0000         1
          14     0.0000    0.0000    0.0000         1
          15     0.0000    0.0000    0.0000         1
  


/root/miniconda3/envs/cre/lib/python3.10/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
/root/miniconda3/envs/cre/lib/python3.10/site-packages/sklearn/utils/multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  ys_types = set(type_of_target(x) for x in ys)
/root/miniconda3/envs/cre/lib/python3.10/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
/root/miniconda3/envs/cre/lib/python3.10/site-packages/sklearn/utils/multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  ys_types = set(type_of_target(x) for x in ys)
/root/miniconda3/envs/cre/lib/python3.10/site-packages/sklearn/metrics/_c

In [9]:
def inference_and_report(model, dataloader, criterion, device, 
                         pred_class_names, gt_class_names, 
                         num_samples_to_print=20):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Inference"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(dataloader.dataset)
    
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
    print(f"\nInference Results → Loss: {avg_loss:.4f}, Accuracy (by index): {accuracy:.4f}\n")

    # --- In ra dự đoán và ground truth với 2 bộ tên lớp riêng biệt ---
    print("\n=== Sample Predictions vs. Ground Truth ===")
    num_to_show = min(len(all_preds), num_samples_to_print)
    print(f"Showing first {num_to_show} samples:\n")
    print(f"{'Sample':<10} | {'Prediction Class':<30} | {'Ground Truth Class':<30}")
    print("-" * 75)
    for i in range(num_to_show):
        # Sử dụng đúng tệp class_names cho từng loại nhãn
        pred_class_name = pred_class_names[all_preds[i]]
        label_class_name = gt_class_names[all_labels[i]]
        print(f"{i+1:<10} | {pred_class_name:<30} | {label_class_name:<30}")


PRED_CLASS_NAMES_PATH = '/workspace/data/npy_splits_new/class_names.npy'
GT_CLASS_NAMES_PATH = '/workspace/data/npy_splits_new/class_names.npy' 

try:
    pred_class_names = np.load(PRED_CLASS_NAMES_PATH, allow_pickle=True)
    gt_class_names = np.load(GT_CLASS_NAMES_PATH, allow_pickle=True)
except FileNotFoundError as e:
    print(f"Lỗi: Không tìm thấy tệp .npy. Vui lòng kiểm tra lại đường dẫn.")
    print(f"Chi tiết lỗi: {e}")
    exit()

# 2. Gọi hàm inference và truyền cả 2 danh sách tên lớp vào
# Giả sử model, test_loader, criterion, device đã được định nghĩa
inference_and_report(
    model, 
    test_loader, 
    criterion, 
    device, 
    pred_class_names=pred_class_names, 
    gt_class_names=gt_class_names, 
    num_samples_to_print=50
)

Inference: 100%|██████████| 9/9 [00:00<00:00, 50.04it/s]


Inference Results → Loss: 92.6517, Accuracy (by index): 0.2950


=== Sample Predictions vs. Ground Truth ===
Showing first 50 samples:

Sample     | Prediction Class               | Ground Truth Class            
---------------------------------------------------------------------------
1          | ngày_nhà_giáo_việt_namN        | 7                             
2          | rong_chơiB                     | a                             
3          | lung_tungT                     | aiB                           
4          | vấn_đềB                        | aiT                           
5          | bãi_đỗ_xe_ô_tô                 | an_giang                      
6          | tuần_nàyT                      | anh_dũngB                     
7          | tạo_dáng                       | anh_emB                       
8          | mái_tóc                        | anh_hai_anh_cảB               
9          | anh_hai_anh_cảN                | anh_hai_anh_cảN               
10         | cháu

In [10]:
def inference_and_report(model, dataloader, criterion, device, 
                         pred_class_names, gt_class_names, 
                         print_range=(0, 15)): # <-- Tham số mới
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Inference"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(dataloader.dataset)
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
    print(f"\nInference Results → Loss: {avg_loss:.4f}, Accuracy (by index): {accuracy:.4f}\n")

    # --- PHẦN ĐƯỢC CẬP NHẬT: In theo khoảng ---
    print("\n=== Sample Predictions vs. Ground Truth ===")
    
    start_idx, end_idx = print_range
    total_samples = len(all_preds)

    # Kiểm tra để đảm bảo khoảng in hợp lệ
    if not (0 <= start_idx < end_idx <= total_samples):
        print(f"Lỗi: Khoảng in [{start_idx}, {end_idx}) không hợp lệ. Tổng số mẫu là {total_samples}.")
        # In 15 mẫu đầu tiên như mặc định nếu có lỗi
        start_idx, end_idx = 0, min(15, total_samples)
        
    print(f"Showing samples from index {start_idx} to {end_idx - 1}:\n")
    print(f"{'Sample #':<10} | {'Prediction Class':<30} | {'Ground Truth Class':<30}")
    print("-" * 75)

    # Lặp qua đúng khoảng chỉ số đã cho
    for i in range(start_idx, end_idx):
        pred_class_name = pred_class_names[all_preds[i]]
        label_class_name = gt_class_names[all_labels[i]]
        print(f"{i + 1:<10} | {pred_class_name:<30} | {label_class_name:<30}")

PRED_CLASS_NAMES_PATH = '/workspace/data/npy_splits_new/class_names.npy'
GT_CLASS_NAMES_PATH = '/workspace/data/npy_splits_new/class_names.npy' 

try:
    pred_class_names = np.load(PRED_CLASS_NAMES_PATH, allow_pickle=True)
    gt_class_names = np.load(GT_CLASS_NAMES_PATH, allow_pickle=True)
except FileNotFoundError as e:
    print(f"Lỗi: Không tìm thấy tệp .npy. Vui lòng kiểm tra lại đường dẫn.")
    print(f"Chi tiết lỗi: {e}")
    exit()

a = 0

inference_and_report(
    model, 
    test_loader, 
    criterion, 
    device, 
    pred_class_names=pred_class_names, 
    gt_class_names=gt_class_names, 
    print_range=(a, a + 15)
)

Inference: 100%|██████████| 9/9 [00:00<00:00, 49.13it/s]


Inference Results → Loss: 92.6517, Accuracy (by index): 0.2950


=== Sample Predictions vs. Ground Truth ===
Showing samples from index 0 to 14:

Sample #   | Prediction Class               | Ground Truth Class            
---------------------------------------------------------------------------
1          | ngày_nhà_giáo_việt_namN        | 7                             
2          | rong_chơiB                     | a                             
3          | lung_tungT                     | aiB                           
4          | vấn_đềB                        | aiT                           
5          | bãi_đỗ_xe_ô_tô                 | an_giang                      
6          | tuần_nàyT                      | anh_dũngB                     
7          | tạo_dáng                       | anh_emB                       
8          | mái_tóc                        | anh_hai_anh_cảB               
9          | anh_hai_anh_cảN                | anh_hai_anh_cảN               
10     